## 박스오피스 EDA

- 응답구조
    - boxofficeType :문자열	박스오피스 종류
    - showRange : 	문자열	박스오피스 조회 일자를 출력
    - rnum	문자열	순번을 출력
    - rank	문자열	해당일자의 박스오피스 순위를 출력
    - rankInten	문자열	전일대비 순위의 증감분을 출력
    - rankOldAndNew	문자열	랭킹에 신규진입여부를 출력
    - “OLD” : 기존 , “NEW” : 신규
    - movieCd: 문자열	영화의 대표코드를 출력
    - movieNm : 문자열	영화명(국문)을 출력
    - openDt : 문자열	영화의 개봉일을 출력
    - salesAmt : 문자열	해당일의 매출액을 출력
    - salesShare : 문자열	해당일자 상영작의 매출총액 대비 해당 영화의 매출비율을 출력
    - salesInten : 문자열	전일 대비 매출액 증감분을 출력
    - salesChange : 문자열	전일 대비 매출액 증감 비율을 출력
    - salesAcc : 문자열	누적매출액을 출력
    - audiCnt : 문자열	해당일의 관객수를 출력
    - audiInten : 문자열	전일 대비 관객수 증감분을 출력
    - audiChange : 문자열	전일 대비 관객수 증감 비율을 출력
    - audiAcc : 문자열	누적관객수를 출력합니다.
    - scrnCnt : 문자열	해당일자에 상영한 스크린수를 출력합
    - showCnt : 문자열	해당일자에 상영된 횟수를 출력

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('boxoffice_20_23.csv')
df.head()

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,current_date
0,1,1,0,OLD,20192206,백두산,2019-12-19,4430968920,37.3,1900868950,75.1,57336758690,498802,182608,57.8,6789304,1236,5845,2020-01-01
1,2,2,0,OLD,20184571,천문: 하늘에 묻는다,2019-12-26,2092505020,17.6,916870400,78.0,9900747970,239149,89345,59.6,1204773,929,3439,2020-01-01
2,3,3,0,OLD,20198414,시동,2019-12-18,1691272350,14.3,589055650,53.4,22927750150,190455,54187,39.8,2716441,801,2997,2020-01-01
3,4,4,0,OLD,20196272,미드웨이,2019-12-31,1561075860,13.2,489446340,45.7,2646735380,174407,42527,32.2,307780,810,2614,2020-01-01
4,5,5,0,OLD,20197803,겨울왕국 2,2019-11-21,727968600,6.1,288393600,65.6,112324217320,83529,28067,50.6,13452593,665,1459,2020-01-01


In [4]:
df

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,current_date
0,1,1,0,OLD,20192206,백두산,2019-12-19,4430968920,37.3,1900868950,75.1,57336758690,498802,182608,57.8,6789304,1236,5845,2020-01-01
1,2,2,0,OLD,20184571,천문: 하늘에 묻는다,2019-12-26,2092505020,17.6,916870400,78.0,9900747970,239149,89345,59.6,1204773,929,3439,2020-01-01
2,3,3,0,OLD,20198414,시동,2019-12-18,1691272350,14.3,589055650,53.4,22927750150,190455,54187,39.8,2716441,801,2997,2020-01-01
3,4,4,0,OLD,20196272,미드웨이,2019-12-31,1561075860,13.2,489446340,45.7,2646735380,174407,42527,32.2,307780,810,2614,2020-01-01
4,5,5,0,OLD,20197803,겨울왕국 2,2019-11-21,727968600,6.1,288393600,65.6,112324217320,83529,28067,50.6,13452593,665,1459,2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14605,6,6,0,OLD,20235290,도티와 영원의 탑,2023-12-27,113512552,1.2,3649005,3.3,388161976,11972,311,2.7,41696,370,438,2023-12-31
14606,7,7,0,OLD,20234673,뽀로로 극장판 슈퍼스타 대모험,2023-12-13,111402091,1.2,10830582,10.8,2933239296,11396,930,8.9,321190,326,424,2023-12-31
14607,8,8,0,OLD,20234114,괴물,2023-11-29,96150674,1.1,14102753,17.2,3791952275,9234,1465,18.9,387427,187,266,2023-12-31
14608,9,9,0,OLD,20236045,"바다 탐험대 옥토넛 어보브 앤 비욘드: 버드, 옥토경보를 울려라!",2023-12-27,53802250,0.6,6104390,12.8,319879760,6404,642,11.1,38823,212,254,2023-12-31


In [6]:
data= pd.read_csv('tv_shows.csv')

In [7]:
data

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type
0,0,1,Breaking Bad,2008,18+,9.4/10,100/100,1,0,0,0,1
1,1,2,Stranger Things,2016,16+,8.7/10,96/100,1,0,0,0,1
2,2,3,Attack on Titan,2013,18+,9.0/10,95/100,1,1,0,0,1
3,3,4,Better Call Saul,2015,18+,8.8/10,94/100,1,0,0,0,1
4,4,5,Dark,2017,16+,8.8/10,93/100,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5363,5363,5713,Paradise Islands,2017,NaN,NaN,10/100,0,0,0,1,1
5364,5364,5714,Mexico Untamed,2018,NaN,NaN,10/100,0,0,0,1,1
5365,5365,5715,Wild Centeral America,2020,NaN,NaN,10/100,0,0,0,1,1
5366,5366,5716,Wild Russia,2018,NaN,NaN,10/100,0,0,0,1,1


In [9]:
data.Year.value_counts() # 2020년도 컨텐

2017    605
2018    562
2019    500
2020    489
2016    486
       ... 
1943      1
1932      1
1947      1
1904      1
1945      1
Name: Year, Length: 78, dtype: int64

In [18]:
#df1 = pd.read_csv('all-weeks-countries.csv', encoding='latin1')

In [19]:
#df1.info()